# Monitor and Govern Databricks Workspaces

Use system tables to monitor usage, costs, and implement governance with Unity Catalog.

## What You'll Learn

✅ Query system tables for observability  
✅ Analyze billing and cost allocation  
✅ Monitor workspace usage and performance  
✅ Implement Unity Catalog security  
✅ Create governance dashboards  

**Note**: Since students won't have access to actual system tables, we'll use synthetic data that matches the schema.

---

**References:**
- [System Tables](https://docs.databricks.com/aws/en/admin/system-tables/)
- [Billing Tables](https://docs.databricks.com/aws/en/admin/system-tables/billing)
- [Unity Catalog Governance](https://docs.databricks.com/aws/en/data-governance/unity-catalog/)
- [Observability Dashboards](https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources)

In [0]:
import re

CATALOG = 'dwx_airops_insights_platform_dev_workspace'
READ_SCHEMA = 'db_crash_course'  # Shared schema (read-only)
username = spark.sql("SELECT current_user()").collect()[0][0]
username_base = username.split('@')[0]  # Extract username before @ symbol
WRITE_SCHEMA = re.sub(r'[^a-zA-Z0-9_]', '_', username_base)  # Replace special chars with _

# Create personal schema if it doesn't exist
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{WRITE_SCHEMA}")

print(f"✅ Using catalog: {CATALOG}")
print(f"📖 Reading IoT data from schema: {READ_SCHEMA} (shared)")
print(f"📊 Reading system tables from schema: {WRITE_SCHEMA} (your personal schema)")
print(f"\nNote: System tables (system_billing, query_history, audit_logs) were created")
print(f"      in your personal schema by the setup script.")

## Cost Analysis Queries

Let's analyze our synthetic billing data to understand cost patterns. In a real production environment, you would use `system.billing.*` tables.

**IMPORTANT:** The system tables (system_billing, query_history, audit_logs, user_permissions) are in YOUR personal schema, not a shared "training" schema. The SQL queries below use `{CATALOG}.{WRITE_SCHEMA}` placeholders that you'll need to replace with your actual catalog and schema.

**For SQL cells below**: Replace `{CATALOG}.{WRITE_SCHEMA}` with your values from the configuration cell above.
- Example: If WRITE_SCHEMA is `jane_smith` (extracted from jane.smith@company.com), use `dwx_airops_insights_platform_dev_workspace.jane_smith`

**Or convert to Python**: Use `spark.sql(f"SELECT ... FROM {CATALOG}.{WRITE_SCHEMA}.system_billing ...")`

### Total Cost by Day

In [0]:
# Total Cost by Day
spark.sql(f"""
SELECT 
  usage_date,
  SUM(usage_quantity * list_price) as total_cost,
  COUNT(*) as num_operations
FROM {CATALOG}.{READ_SCHEMA}.system_billing
GROUP BY usage_date
ORDER BY usage_date DESC
LIMIT 10
""").display()

In [0]:
# Cost by Workspace
spark.sql(f"""
SELECT 
  workspace_id,
  SUM(usage_quantity * list_price) as total_cost,
  COUNT(*) as num_operations,
  AVG(usage_quantity * list_price) as avg_cost_per_operation
FROM {CATALOG}.{READ_SCHEMA}.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY workspace_id
ORDER BY total_cost DESC
""").display()

In [0]:
# Cost by SKU Type
spark.sql(f"""
SELECT 
  sku_name,
  SUM(usage_quantity) as total_dbus,
  SUM(usage_quantity * list_price) as total_cost,
  AVG(usage_quantity * list_price) as avg_cost_per_operation,
  COUNT(*) as operations
FROM {CATALOG}.{READ_SCHEMA}.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY sku_name
ORDER BY total_cost DESC
""").display()

In [0]:
# Cost by User
spark.sql(f"""
SELECT 
  usage_metadata.user,
  COUNT(*) as operations,
  SUM(usage_quantity * list_price) as total_cost,
  AVG(usage_quantity * list_price) as avg_cost_per_operation
FROM {CATALOG}.{READ_SCHEMA}.system_billing
WHERE usage_date >= CURRENT_DATE - 30
GROUP BY usage_metadata.user
ORDER BY total_cost DESC
LIMIT 10
""").display()

In [0]:
# Consolidated query performance summary
spark.sql(f"""
SELECT 
  query_type,
  COUNT(*) as query_count,
  AVG(execution_time_ms) as avg_duration_ms,
  AVG(compute_cost) as avg_cost,
  SUM(compute_cost) as total_cost
FROM {CATALOG}.{READ_SCHEMA}.query_history
WHERE query_start_time >= CURRENT_DATE - 7
GROUP BY query_type
ORDER BY total_cost DESC
""").display()

In [0]:
spark.sql(f"""
-- Consolidated query performance summary
SELECT 
  query_type,
  COUNT(*) as query_count,
  AVG(execution_time_ms) as avg_duration_ms,
  AVG(compute_cost) as avg_cost,
  SUM(compute_cost) as total_cost
FROM {CATALOG}.{READ_SCHEMA}.query_history
WHERE query_start_time >= CURRENT_DATE - 7
GROUP BY query_type
ORDER BY total_cost DESC;
""")

**Key Insights:**
- Identify which query types are driving costs
- Spot performance optimization opportunities
- Track usage patterns across your team

For more detailed analysis, explore the [Observability Dashboards](https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources) examples.


### Use Cases for AI Forecast

**Cost Management:**
- Predict monthly spending for budget planning
- Identify cost spikes before they happen
- Allocate resources based on forecasted demand

**Capacity Planning:**
- Forecast compute usage by team/project
- Plan infrastructure scaling
- Optimize reserved capacity purchases

**Reference:** [AI Forecast Documentation](https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_forecast)

**💡 Pro Tip:** Create a scheduled job that runs these forecasts daily and sends alerts when predicted costs exceed thresholds.



In [0]:
%sql
-- Forecast daily costs for the next 7 days using AI
-- Note: This requires a SQL Warehouse connection

SELECT 
  usage_date,
  total_cost,
  forecast,
  lower_bound,
  upper_bound
FROM (
  SELECT 
    ai_forecast(
      (SELECT usage_date, SUM(usage_quantity * list_price) as total_cost
       FROM {CATALOG}.{READ_SCHEMA}
       GROUP BY usage_date
       ORDER BY usage_date),
      horizon => 7
    )
  )
ORDER BY usage_date DESC


# Unity Catalog Access Management

One of the most important aspects of data governance is controlling who can access what data. Unity Catalog provides fine-grained access control at multiple levels: catalogs, schemas, tables, columns, and even rows.

Let's walk through a practical example of setting up access controls for our IoT sensor data.


### Step 1: Create Groups

Groups make it easier to manage permissions at scale. Instead of granting access to individual users, you grant it to groups.

**Common groups for an IoT project:**
- `data_engineers` - Can read/write raw and processed data
- `data_analysts` - Can read processed data and create dashboards
- `ml_engineers` - Can read data and create/deploy ML models
- `executives` - Read-only access to dashboards and reports


In [0]:
%sql
-- Create a group for data analysts
-- Note: In Databricks, groups are typically managed at the account level
-- These commands would be run by an account admin

CREATE GROUP IF NOT EXISTS `data_analysts_xyz`; -- You will hit an error later if you don't create your own group


### Step 2: Add Users to Groups

Once groups are created, you can add users to them. Users inherit all permissions granted to the groups they belong to.


In [0]:
%sql
-- Add users to groups
-- Replace with actual user emails from your organization

ALTER GROUP `data_analysts_xyz` ADD USER `jane.smith@company.com`; --make sure you use your own group created above, and add a real user

### Step 3: Grant Table Access

Now let's grant the appropriate permissions on our IoT tables. Unity Catalog uses a hierarchical permission model:

**Hierarchy:** Catalog → Schema → Table

**Permission Types:**
- `USE CATALOG` - Required to see/access a catalog
- `USE SCHEMA` - Required to see/access a schema
- `SELECT` - Read data from tables
- `MODIFY` - Update/delete data
- `CREATE TABLE` - Create new tables in a schema
- `ALL PRIVILEGES` - Full control

Let's grant permissions for our sensor and inspection data:


In [0]:
%sql
-- First, grant catalog and schema level permissions
-- Data analysts need to USE the catalog and schema to access tables
GRANT USE CATALOG ON CATALOG dwx_airops_insights_platform_dev_workspace TO `data_analysts_xyz`;
GRANT USE SCHEMA ON SCHEMA dwx_airops_insights_platform_dev_workspace.{your_username} TO `data_analysts_xyz`;
GRANT SELECT ON TABLE dwx_airops_insights_platform_dev_workspace.{your_username}.sensor_bronze TO `data_analysts_xyz`;

### Step 5: View Current Permissions

You can check what permissions have been granted using the `SHOW GRANTS` command:


In [0]:
%sql
-- View all grants on a specific table
SHOW GRANTS ON TABLE dwx_airops_insights_platform_dev_workspace.{your_username}.sensor_bronze;

-- View all grants for a specific group
-- SHOW GRANTS TO `data_analysts`;

-- View grants on an entire schema
-- SHOW GRANTS ON SCHEMA training.default;


### Revoking Permissions

If you need to remove access, use the `REVOKE` command:

```sql
-- Revoke SELECT on a specific table
REVOKE SELECT ON TABLE training.sensor_bronze FROM `data_analysts`;

-- Revoke all privileges on a schema
REVOKE ALL PRIVILEGES ON SCHEMA training.default FROM `data_analysts`;
```

**Best Practices for Access Management:**

1. **Use groups, not individual users** - Makes management much easier
2. **Grant minimum necessary permissions** - Start with read-only, add write permissions only when needed
3. **Use separate schemas for different purposes** - e.g., `raw`, `processed`, `ml_features`, `production`
4. **Document your permission model** - Keep track of which groups have access to what
5. **Regular audits** - Review permissions quarterly to ensure they're still appropriate
6. **Use service principals for automation** - Don't use personal accounts for scheduled jobs


### Grant Privileges (Examples)

**Note:** These are example commands. In production, you would grant privileges to actual user groups.

**Grant SELECT on schema:**
```sql
GRANT SELECT ON SCHEMA training TO `data-analysts`;
```

**Grant table access:**
```sql
GRANT SELECT ON TABLE training.system_billing TO `data-analysts`;
```

**Grant usage on catalog:**
```sql
GRANT USAGE ON CATALOG <your_catalog> TO `data-analysts`;
```

### Row-Level Security (Conceptual Example)

Unity Catalog supports row filters to restrict data based on user permissions. Here's how it works:

**Step 1: Create a filter function**
```sql
CREATE FUNCTION training.filter_by_region(region STRING)
RETURN region IN (
  SELECT region FROM training.user_permissions 
  WHERE user_email = current_user()
);
```

**Step 2: Apply the filter to a table**
```sql
ALTER TABLE <your_table>
SET ROW FILTER training.filter_by_region(region) ON (region);
```

This ensures users only see data for their authorized regions.

### Column Masking (Conceptual Example)

Mask sensitive columns based on user roles:

**Step 1: Create masking function**
```sql
CREATE FUNCTION training.mask_device_id(device_id STRING)
RETURN CASE 
  WHEN is_member('admin') THEN device_id
  ELSE CONCAT('***', RIGHT(device_id, 4))
END;
```

**Step 2: Apply mask to column**
```sql
ALTER TABLE <your_table>
ALTER COLUMN device_id
SET MASK training.mask_device_id;
```

Non-admin users will only see masked device IDs (e.g., "***1234").



---

**Additional Resources:**
- [System Tables Guide](https://docs.databricks.com/aws/en/admin/system-tables/)
- [Unity Catalog Security](https://docs.databricks.com/aws/en/data-governance/unity-catalog/access-control)
- [Observability Examples](https://github.com/CodyAustinDavis/dbsql_sme/tree/main/Observability%20Dashboards%20and%20DBA%20Resources)
- [Cost Management](https://docs.databricks.com/aws/en/admin/account-settings/usage-detail-tags-aws)
- [Row Filters and Column Masks](https://docs.databricks.com/aws/en/data-governance/unity-catalog/row-and-column-filters)
- [AI Forecast Function](https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_forecast)

---

**🎉 You've completed Day 3!** You now have the skills to build end-to-end data and ML pipelines, monitor costs, forecast future spending with AI, and govern your Databricks workspace effectively.